# Install Seldon Core in Kind using Ansible

This runbook illustrates how Seldon Core (including Istio and MinIO) can be installed newly created Kind cluster.

## Install Required Dependencies

As per documentation for our [ansible-k8s-collection](https://github.com/SeldonIO/ansible-k8s-collection) we will require a following

```yaml
pip install ansible openshift docker passlib
ansible-galaxy collection install git+https://github.com/SeldonIO/ansible-k8s-collection.git
```

## Created local configuration files

In [2]:
%%writefile inventory.ini
[all:vars]
ansible_python_interpreter=python3

[all]
localhost ansible_connection=local

Overwriting inventory.ini


In [3]:
%%writefile ansible.cfg
[defaults]
inventory = ./inventory.ini
nocows = 1

Overwriting ansible.cfg


## Create and execute Ansible Playbook

In [15]:
%%writefile core.yaml
---
- name: Install Seldon Core
  hosts: localhost
  roles:
    - seldonio.k8s.kind
    - seldonio.k8s.metallb    
    - seldonio.k8s.istio
    - seldonio.k8s.seldon_core
  vars:
    kind_cluster_name: ansible
    kind_image_version: kindest/node:v1.18.15
    kind_kubectl_default_namespace: seldon
        
    istio_version: 1.7.6
    istio_verify_install: false
    seldon_core_version: v1.9.1
    seldon_core_values:
      istio:
        enabled: true
        gateway: "seldon-gateway.istio-system.svc.cluster.local"

Overwriting core.yaml


In [16]:
!ansible-playbook core.yaml

[DEPRECATION WARNING]: Ansible will require Python 3.8 or newer on the 
controller starting with Ansible 2.12. Current version: 3.7.10 | packaged by 
conda-forge | (default, Feb 19 2021, 16:07:37) [GCC 9.3.0]. This feature will 
be removed from ansible-core in version 2.12. Deprecation warnings can be 
disabled by setting deprecation_warnings=False in ansible.cfg.

PLAY [Install Seldon Core] *****************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [seldonio.k8s.kind : Check if KinD Cluster already exists: ansible] *******
ok: [localhost]

TASK [seldonio.k8s.kind : Start KinD Cluster: 'ansible'] ***********************
skipping: [localhost]

TASK [seldonio.k8s.kind : Echo output of "kind create cluster ..." command] ****
skipping: [localhost]

TASK [seldonio.k8s.kind : Export kubeconfig for KinD Cluster: 'ansible'] *******
changed: [localhost]

TASK [seldonio.k8s.kind : Create a k8s name

## Verify the installation

In [17]:
!kind get clusters

ansible


In [28]:
!kubectl get pods -A

NAMESPACE            NAME                                            READY   STATUS        RESTARTS   AGE
istio-system         istio-ingressgateway-797b6b595c-t47lb           1/1     Running       0          176m
istio-system         istiod-55c94c6799-blmrw                         1/1     Running       0          177m
kube-system          coredns-66bff467f8-dmjnf                        1/1     Running       0          177m
kube-system          coredns-66bff467f8-spkj7                        1/1     Running       0          177m
kube-system          etcd-ansible-control-plane                      1/1     Running       0          177m
kube-system          kindnet-qcq5t                                   1/1     Running       0          177m
kube-system          kube-apiserver-ansible-control-plane            1/1     Running       0          177m
kube-system          kube-controller-manager-ansible-control-plane   1/1     Running       0          177m
kube-system          kube-proxy-n2gqn 

## Create and test deployment

In [25]:
%%writefile sklearn.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  predictors:
    - graph:
        implementation: SKLEARN_SERVER
        modelUri: gs://seldon-models/sklearn/iris
        name: classifier
      name: default
      replicas: 1

Overwriting sklearn.yaml


In [26]:
!kubectl apply -f sklearn.yaml

seldondeployment.machinelearning.seldon.io/sklearn unchanged


In [27]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

deployment "sklearn-default-0-classifier" successfully rolled out


With MetalLB Istio Ingress will have a local IP

In [29]:
!kubectl get svc -n istio-system

NAME                   TYPE           CLUSTER-IP      EXTERNAL-IP    PORT(S)                                                      AGE
istio-ingressgateway   LoadBalancer   10.96.31.162    172.18.255.1   15021:30852/TCP,80:31393/TCP,443:32669/TCP,15443:31281/TCP   176m
istiod                 ClusterIP      10.96.249.126   <none>         15010/TCP,15012/TCP,443/TCP,15014/TCP,853/TCP                177m


In [31]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://172.18.255.1/seldon/seldon/sklearn/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364169,
        0.04505474761561406,
        5.7927447968952436e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.9.1"
    }
  }
}


## Cleanup

In [32]:
!kind delete cluster --name ansible

Deleting cluster "ansible" ...
